In [ ]:
#!pip3 install zstandard pandas
#!pip3 install "dask[complete]"

In [1]:
import json
import pandas as pd
import numpy as np 
from datetime import datetime,timezone
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import zstandard
import os
import sys
import logging.handlers
import re
from tqdm import tqdm
import csv
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import re

In [3]:
#reading posts 
file_path = "/Users/jasleenmadan/Documents/Workspace/Data/reddit/wallstreetbets_submissions"
start_time = 1606780800  # 1 Dec
end_time =  1613462399 # 15 Feb

list_reddit = []
    
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line:
         r_full = json.loads(line)
         created_utc = int(r_full.get('created_utc'))
         if start_time <= created_utc <= end_time:
          r = {key: r_full[key] for key in r_full.keys()
             & {'id','created_utc','title','num_comments','selftext','score','distinguished','upvote_ratio'}}
          list_reddit.append(r)
                   

In [9]:
post_df = pd.DataFrame(list_reddit)
print(post_df.shape)
#post_df.to_csv('posts_raw_15_09_24.csv', index=False)

(894803, 8)


In [3]:
#reading in comments dataset
file_path_com = "/Users/jasleenmadan/Documents/Workspace/Data/reddit/wallstreetbets_comments"
start_time = 1606780800  # 1 Dec 
end_time =  1613462399 # 28 Feb

out_csv = []
skip_phrases = {'[deleted]', '[removed]', '[deleted by user]'}
    
with open(file_path_com, 'r') as file:
    for line in file:
        line = line.strip()
        if line:
            json_obj = json.loads(line)
            created_utc = int(json_obj.get('created_utc'))
            body = json_obj.get('body', '').strip()
            if start_time <= created_utc <= end_time and not any(phrase in body for phrase in skip_phrases):   
                r = {key: json_obj[key] for key in json_obj.keys() 
                        & {'created_utc', 'author_flair_type','distinguished','score', 'body', 'id', 'parent_id', 'link_id'}}
                out_csv.append(r)

In [4]:
comments_df = pd.DataFrame(out_csv)
comments_df.to_csv('comments_raw_15_09_24.csv', index=False)